# 数据预处理

In [1]:
import pandas as pd
from dateutil.parser import parse
import numpy as np
%matplotlib inline

dateparse = lambda dates: pd.datetime.strptime(dates,'%Y-%m-%d')
data = pd.read_csv('./data/江苏.csv' ,dtype={'ORG':object, 'SPEC_CODE':object, 'COMM_MODE':object, 'MANUFACTURER':object,
                                                   'FAULT_QUARTER':object}, date_parser=dateparse)
data.head()

,"﻿""EQUIP_ID""",FAULT_TYPE,INST_DATE,DETECT_DATE,FAULT_DATE,SYNC_ORG_NO,ORG_NO,ORG_NAME,SORT_CODE,SPEC_CODE,COMM_MODE,ARRIVE_BATCH_NO,MANUFACTURER
0,270012379016,40401,2013-12-20,2012-12-18,2015-9-18 上午 10:20:28,32101,324120632,戴窑供电所,10,102,01,111122050041114,10010
1,300010573131,40501,2016-8-1,2013-12-3,2015-7-13 上午 08:45:36,32101,324120632,戴窑供电所,10,102,01,111124130051127,00429
2,270008603828,20106,2012-10-23,2012-7-27,2015-2-4 上午 08:50:24,32101,324120632,戴窑供电所,10,102,01,111115040080717,00572
3,220005003818,41201,2011-11-14,2011-10-8,2015-1-16 下午 02:19:48,32101,324120632,戴窑供电所,10,102,01,220000022702,00588
4,220005010515,41201,2011-10-30,2011-9-30,2015-1-16 下午 02:18:15,32101,324120632,戴窑供电所,10,102,01,220000022702,00588


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171705 entries, 0 to 171704
Data columns (total 13 columns):
﻿"EQUIP_ID"        171705 non-null int64
FAULT_TYPE         171705 non-null int64
INST_DATE          171179 non-null object
DETECT_DATE        171221 non-null object
FAULT_DATE         171705 non-null object
SYNC_ORG_NO        171705 non-null int64
ORG_NO             171705 non-null int64
ORG_NAME           171705 non-null object
SORT_CODE          171705 non-null int64
SPEC_CODE          171705 non-null object
COMM_MODE          161699 non-null object
ARRIVE_BATCH_NO    171705 non-null object
MANUFACTURER       171705 non-null object
dtypes: int64(5), object(8)
memory usage: 18.3 MB


In [3]:
#转化日期格式
data['FAULT_DATE'] = pd.to_datetime(data['FAULT_DATE'].str.strip().str.split(' ').str[0])
data['INST_DATE'] = pd.to_datetime(data['INST_DATE'].str.strip().str.split(' ').str[0])

In [4]:
data['FAULT_MONTH'] = [x.month for x in data['FAULT_DATE']]
data['INST_MONTH'] = [x.month for x in data['INST_DATE']]

In [5]:
#计算使用时间
print sum(data['FAULT_DATE'].isnull()), sum(data['INST_DATE'].isnull())
data['month'] = data['FAULT_DATE'] - data['INST_DATE']
data['month'].fillna(method='ffill', inplace=True)
data['month'] = [x.days/30 for x in data['month']]
data['month'][data['month']<0] = int(data['month'][data['month']>0].mean())   #用均值补充小于0的值

0 526


/home/hadoop/.local/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
#筛选数据
data = data[data['SORT_CODE']==10] #选出SORT_CODE 为10 的智能表
data['FAULT_TYPE'] = [x[:3] for x in data['FAULT_TYPE'].values.astype('str')]
data = data[(data['FAULT_TYPE']=='402') | (data['FAULT_TYPE']=='405') | (data['FAULT_TYPE']=='406') | (data['FAULT_TYPE']=='403')]
data['FAULT_TYPE'].value_counts()
data['ORG_NO'] = [x[:5] for x in data['ORG_NO'].values.astype('str')]

In [7]:
#删除无用数据
data.drop(['DETECT_DATE', 'SYNC_ORG_NO', 'ORG_NAME', 'SORT_CODE', 'ARRIVE_BATCH_NO', 'FAULT_DATE', 'INST_DATE', 'DETECT_DATE'], 
         axis=1, inplace=True)

In [12]:
data['COMM_MODE'].value_counts()
data.drop(['COMM_MODE'], axis =1 , inplace=True)

In [14]:
data

,"﻿""EQUIP_ID""",FAULT_TYPE,ORG_NO,SPEC_CODE,MANUFACTURER,FAULT_MONTH,INST_MONTH,month
1,300010573131,405,32412,102,00429,7,8,32
32,270002987725,406,32408,102,00268,1,4,33
53,270002779237,402,32406,102,00495,1,3,34
62,110005878060,402,32401,102,10001,1,2,46
167,220004643292,402,32412,102,00845,1,3,46
168,180003807076,406,32408,102,00832,1,10,38
176,300011040286,402,32407,102,00417,1,3,9
212,270007239777,402,32408,102,10001,1,6,31
221,270014017647,405,32412,102,10007,1,4,21
334,270007086477,405,32409,102,00965,1,7,30


In [ ]:
data.to_csv('./data/jiangsu_4.csv', index=False)